# **Deep Learning Computation**

**1. Layers and Block**

In [1]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.02033284, -0.02384312, -0.20464043, -0.1263061 ,  0.05745346,
         0.34566107,  0.00883594,  0.1355887 ,  0.37181625,  0.36286741],
       [-0.0235171 , -0.03886639, -0.45638585, -0.04157028,  0.13332711,
         0.31121284,  0.09139524,  0.28973734,  0.12392002,  0.4365357 ]],
      dtype=float32)>

1.1 Custom Block

In [5]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)
    def call(self, X):
        return self.out(self.hidden((X)))

In [6]:
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.0948078 ,  0.24992228, -0.1590398 ,  0.03518994,  0.28906447,
        -0.19760886,  0.0225349 ,  0.21806619, -0.13676386,  0.0352051 ],
       [-0.10221779,  0.07890572, -0.07550462,  0.33047658,  0.08063342,
        -0.02278177, -0.07280055,  0.15085572,  0.00908687, -0.05547542]],
      dtype=float32)>

1.2 The Sequential Block

In [7]:
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for block in args:
            # Here, `block` is an instance of a `tf.keras.layers.Layer`
            # subclass
            self.modules.append(block)

    def call(self, X):
        for module in self.modules:
            X = module(X)
        return X

In [8]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.10432345, -0.12084252, -0.02400047, -0.17608589, -0.09269628,
         0.10795489, -0.02927341, -0.5174448 , -0.23034364, -0.10060762],
       [-0.07805066, -0.14270982,  0.16359629, -0.3703683 , -0.10993281,
         0.28001106, -0.12852353, -0.1608432 , -0.04418997, -0.21185318]],
      dtype=float32)>

1.3 Executing Code in The Forward Propagation Function

In [9]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(tf.random.uniform((20, 20)))
        self.dense = tf.keras.layers.Dense(20, activation=tf.nn.relu)

    def call(self, inputs):
        X = self.flatten(inputs)
        X = tf.nn.relu(tf.matmul(X, self.rand_weight) + 1)
        X = self.dense(X)
        while tf.reduce_sum(tf.math.abs(X)) > 1:
            X /= 2
        return tf.reduce_sum(X)

In [10]:
net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.90105516>

In [11]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)

    def call(self, inputs):
        return self.dense(self.net(inputs))

chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20))
chimera.add(FixedHiddenMLP())
chimera(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5952771>

# **2. Parameter Management**

In [12]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.26741803],
       [0.40120324]], dtype=float32)>

2.1 Parameter Access

In [13]:
print(net.layers[2].weights)

[<tf.Variable 'dense_13/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[0.04953098],
       [0.39574337],
       [0.2673322 ],
       [0.9926119 ]], dtype=float32)>, <tf.Variable 'dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


2.1.1 Targeted parameters

In [14]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


In [15]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'dense_12/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.44600707,  0.21102661,  0.2930383 , -0.5377409 ],
       [ 0.21936625,  0.32913667, -0.6047634 ,  0.7387952 ],
       [-0.1762296 ,  0.84480995,  0.23226696, -0.08782375],
       [ 0.46524698, -0.27407533, -0.10384178, -0.19128174]],
      dtype=float32)>, <tf.Variable 'dense_12/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[ 0.44600707,  0.21102661,  0.2930383 , -0.5377409 ],
       [ 0.21936625,  0.32913667, -0.6047634 ,  0.7387952 ],
       [-0.1762296 ,  0.84480995,  0.23226696, -0.08782375],
       [ 0.46524698, -0.27407533, -0.10384178, -0.19128174]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[0.04953098],
       [0.39574337],
       [0.2673322 ],
       [0.9926119 ]], dtype=float32), array([0.], dtype=float32)]


In [16]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

2.1.3 Collecting Parameters from Nested Blocks

In [17]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # Nested here
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.09253305],
       [0.17469989]], dtype=float32)>

In [18]:
print(rgnet.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (2, 4)                    80        
                                                                 
 dense_18 (Dense)            (2, 1)                    5         
                                                                 
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'dense_15/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

2.2 Parameters Initialization

2.2.1 Built-in Initialization

In [20]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_19/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.00687757,  0.00502471,  0.00753744, -0.02056741],
        [-0.0138526 , -0.01687078,  0.01199934, -0.0091607 ],
        [-0.00637518,  0.00897097, -0.00251401, -0.0051732 ],
        [ 0.00997718, -0.00408381, -0.02246276,  0.00090009]],
       dtype=float32)>,
 <tf.Variable 'dense_19/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [21]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_21/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_21/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [22]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(42)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'dense_23/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.3705209 ,  0.36428553,  0.07627267, -0.01112145],
       [-0.68365765,  0.863336  ,  0.4518494 , -0.60243416],
       [-0.53056645,  0.09040552,  0.7859685 ,  0.28471965],
       [-0.23487389,  0.08197904, -0.14429533,  0.2748583 ]],
      dtype=float32)>
<tf.Variable 'dense_24/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[42.],
       [42.],
       [42.],
       [42.]], dtype=float32)>


2.2.2 Custom Initialization

In [23]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_25/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 7.217537 , -0.       ,  7.5134563, -0.       ],
       [ 0.       , -8.064297 ,  0.       ,  8.1307335],
       [ 0.       ,  8.901356 , -0.       , -0.       ],
       [-9.025812 , -7.4154925,  0.       , -6.074741 ]], dtype=float32)>


In [24]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'dense_25/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.       ,  1.       ,  8.513456 ,  1.       ],
       [ 1.       , -7.0642967,  1.       ,  9.1307335],
       [ 1.       ,  9.901356 ,  1.       ,  1.       ],
       [-8.025812 , -6.4154925,  1.       , -5.074741 ]], dtype=float32)>

2.3 Tied parameters

In [25]:
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# Check whether the parameters are different
print(len(net.layers) == 3)

True


# **3. Deffered Initialization**

3.1 Instantiating a Network

In [26]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [27]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [28]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

# **4. Custom Layers**

4.1 Layers without Parameters

In [29]:
import tensorflow as tf


class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

In [30]:
layer = CenteredLayer()
layer(tf.constant([1, 2, 3, 4, 5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

In [31]:
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenteredLayer()])

In [32]:
Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=1.8626451e-09>

4.2 Layers with Parameters

In [33]:
class MyDense(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

In [34]:
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()


[array([[-0.05018666,  0.0309556 ,  0.02925599],
        [-0.02281049, -0.0153303 ,  0.02628748],
        [ 0.06903615,  0.01617797, -0.02294693],
        [ 0.03957478, -0.07012694, -0.02378306],
        [ 0.09028004, -0.0489617 , -0.066375  ]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [35]:
# We can directly carry out forward propagation calculations using custom layers.
dense(tf.random.uniform((2, 5)))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.10502525, 0.        , 0.        ],
       [0.04798785, 0.        , 0.        ]], dtype=float32)>

In [36]:
net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.],
       [0.]], dtype=float32)>

# **5. File I/O**

5.1 Loading and Saving Tensors

In [37]:
import numpy as np
import tensorflow as tf

x = tf.range(4)
np.save('x-file.npy', x)

In [38]:
x2 = np.load('x-file.npy', allow_pickle=True)
x2

array([0, 1, 2, 3], dtype=int32)

In [39]:
y = tf.zeros(4)
np.save('xy-files.npy', [x, y])
x2, y2 = np.load('xy-files.npy', allow_pickle=True)
(x2, y2)

(array([0., 1., 2., 3.]), array([0., 0., 0., 0.]))

In [40]:
mydict = {'x': x, 'y': y}
np.save('mydict.npy', mydict)
mydict2 = np.load('mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

5.2 Loading and Saving Model Parameters

In [41]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

net = MLP()
X = tf.random.uniform((2, 20))
Y = net(X)

In [42]:
net.save_weights('mlp.params')

In [43]:
clone = MLP()
clone.load_weights('mlp.params')

In [44]:
Y_clone = clone(X)
Y_clone == Y

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

# **6. GPUs**

In [46]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



6.1 Computing Devices


In [47]:
import tensorflow as tf

tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f1b0d518f00>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7f1b0d5183c0>)

In [48]:
len(tf.config.experimental.list_physical_devices('GPU'))

0

In [49]:
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i + 1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

def try_all_gpus():
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))
    devices = [tf.device(f'/GPU:{i}') for i in range(num_gpus)]
    return devices if devices else [tf.device('/CPU:0')]

try_gpu(), try_gpu(10), try_all_gpus()

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f1b0d51c2d0>,
 [<tensorflow.python.eager.context._EagerDeviceContext at 0x7f1b0d5987d0>])

6.2 Tensors and GPUs

In [50]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:CPU:0'

6.2.1 Storage on the GPU

In [51]:
with try_gpu():
    X = tf.ones((2, 3))
X

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [52]:
with try_gpu(1):
    Y = tf.random.uniform((2, 3))
Y

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.31005168, 0.9861666 , 0.19874144],
       [0.49737835, 0.24993241, 0.8913387 ]], dtype=float32)>

6.2.2 Copying

In [53]:
with try_gpu(1):
    Z = X
print(X)
print(Z)

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)


In [54]:
# Now that the data are on the same GPU (both Z and Y are), we can add them up.
Y + Z

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.3100517, 1.9861666, 1.1987414],
       [1.4973783, 1.2499324, 1.8913387]], dtype=float32)>

In [55]:
with try_gpu(1):
    Z2 = Z
Z2 is Z

True

6.3 Neural Networks and GPUs

In [56]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    net = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1)])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [57]:
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.8378476],
       [-0.8378476]], dtype=float32)>

In [58]:
net.layers[0].weights[0].device, net.layers[0].weights[1].device

('/job:localhost/replica:0/task:0/device:CPU:0',
 '/job:localhost/replica:0/task:0/device:CPU:0')